In [133]:
'''
M1. Integradora


Authors:
Luis Alberto Alcántara Cabrales A01634185
Alexa Serrano Negrete A01654063
Renet de Jesús Pérez Gómez A01640555
Vicente Javier Viera Guízar A01639784

Date:
11/17/2022
'''

#Model
import agentpy as ap
import random


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#Global variable

In [134]:
parameters = {
    'K': 10,
    'citySize': 60,
    'streetSize': 4,
    'steps': 250,
}

In [135]:
class CarAgent(ap.Agent):
    def setup(self):
        self.idTypeOfCar = 0

class StreetAgent(ap.Agent):
    def setup(self):
       self.idTypeOfStreet = 0

In [136]:
def initialPositionCars(cars, street, citySize):
    position = []
    '''
    If modulus is equal to 0, then it going to be in horizontal way
    if not, then it going to be in the vertical way 
    '''

    #Defining the street coordinates
    rightSide = int(((citySize - street) / 2) + 2)
    leftSide = int(rightSide + 1)
    endEnviroment = int(citySize - 1)

    #Create initial positions for the car
    for car in cars: 
        if(car.idTypeOfCar==1):
            position.append((0, rightSide))

        if(car.idTypeOfCar==2):
            position.append((endEnviroment, leftSide))

        if(car.idTypeOfCar==3):
            position.append((rightSide, 0))

        if(car.idTypeOfCar==4):
            position.append((leftSide, endEnviroment))

    return position


def initialPositionStreets(streetSize, citySize):
    position = []

    #Defining the street coordinates
    rightSide = int(((citySize - streetSize) / 2) + 2)
    leftSide = int(rightSide + 1)

    auxCoordinate = 0
    for i in range(citySize):
        position.append((i, rightSide))
        
    for i in range(citySize):
        position.append((i, leftSide))    

    for i in range(citySize):
        position.append((rightSide, i))

    for i in range(citySize):
        position.append((leftSide, i))

    return position

In [137]:
class MyModel(ap.Model):
    def setup(self):
        #Create agents
        self.city = ap.AgentList(self, int(self.p.citySize ** 2))
        self.cars = ap.AgentList(self, self.p.K, CarAgent)
        self.streets = ap.AgentList(self, int(self.p.citySize * 4), StreetAgent)

        #Assing the type of initial position to the cars (where's going to appear for first time)
        for car in self.cars:
            car.idTypeOfCar = random.randint(1,4)

        #Assing the type of street to the streets
        aux = 0
        typeOfStreet = 1
        for street in self.streets:
            if aux <= self.p.citySize:
                street.idTypeOfStreet = typeOfStreet
                aux += 1
            else:
                typeOfStreet += 1
                aux = 0


        #Create grid
        self.area = ap.Grid(self, [self.p.citySize] * 2)
        
        #Determinate the initial position of the cars
        carsPosition = initialPositionCars(self.cars, self.p.streetSize, self.p.citySize)
        
        #Determinate the initial position of the streets
        streetsPosition = initialPositionStreets(self.p.streetSize, self.p.citySize)
        
        #Add agents to enviroment
        self.area.add_agents(self.cars, carsPosition)
        self.area.add_agents(self.streets, streetsPosition)
        
        
        #Defining types of agents 
        self.city.agent_type = 0
        self.cars.agent_type = 1
        self.streets.agent_type = 2
        
    def step(self):
        #Find the center of the simulation 
        rightSide = int(((self.p.citySize - self.p.streetSize) / 2) + 2)
        leftSide = int(rightSide + 1)
        
        for car in self.cars:
            print("Carro: ", car.id, " - idTypeOfCar: ", car.idTypeOfCar)
            print("Coordinate position of the car: ", self.area.positions[car])
            [y, x] = self.area.positions[car]
            neighbors = self.area.neighbors(car)
            print("Numbers of neighbors: ", neighbors)
            
            #Verify if the agent car is in the center
            if y == rightSide and x == rightSide:
                car.idTypeOfCar = random.choice([1,3,4])            
            elif y == rightSide and x == leftSide:
                car.idTypeOfCar = random.choice([2,3,4])
            elif y == leftSide and x == rightSide:
                car.idTypeOfCar = random.choice([1,2,3])
            elif y == leftSide and x == leftSide:
                car.idTypeOfCar = random.choice([1,2,4])

            for neighbor in neighbors:
                print("Tipo de agente: ", neighbor.agent_type)
                if neighbor.agent_type == 2:
                    print("Tipo de calle: ", neighbor.idTypeOfStreet)
                    if car.idTypeOfCar == 1:
                        if neighbor.idTypeOfStreet == 1:
                            self.area.move_by(car, (1,0))
                            break
                            

                    if(car.idTypeOfCar == 2):
                        if neighbor.idTypeOfStreet == 2:
                            self.area.move_by(car, (-1,0))
                            break

                    if(car.idTypeOfCar == 3):
                        if neighbor.idTypeOfStreet == 3:
                            self.area.move_by(car, (0,1))
                            break

                    if(car.idTypeOfCar == 4):
                        if neighbor.idTypeOfStreet == 4:
                            self.area.move_by(car, (0,-1))
                            break 
                # elif neighbor.agent_type == 1:
                #     # print("hola")
                #     if self.area.positions[neighbor] !=  self.area.positions[car]:
                #         break

In [138]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('agent_type')
    color_dict = {0:'#000', 1:'#c1c1c1', 2:'#007', None:'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic Simulation\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))

Carro:  3601  - idTypeOfCar:  2
Coordinate position of the car:  (59, 31)
Numbers of neighbors:  AgentIter (7 objects)
Tipo de agente:  2
Tipo de calle:  1
Tipo de agente:  2
Tipo de calle:  2
Carro:  3602  - idTypeOfCar:  4
Coordinate position of the car:  (31, 59)
Numbers of neighbors:  AgentIter (5 objects)
Tipo de agente:  2
Tipo de calle:  3
Tipo de agente:  2
Tipo de calle:  3
Tipo de agente:  2
Tipo de calle:  4
Carro:  3603  - idTypeOfCar:  3
Coordinate position of the car:  (30, 0)
Numbers of neighbors:  AgentIter (5 objects)
Tipo de agente:  2
Tipo de calle:  2
Tipo de agente:  1
Tipo de agente:  2
Tipo de calle:  2
Tipo de agente:  2
Tipo de calle:  3
Carro:  3604  - idTypeOfCar:  3
Coordinate position of the car:  (30, 0)
Numbers of neighbors:  AgentIter (5 objects)
Tipo de agente:  2
Tipo de calle:  2
Tipo de agente:  1
Tipo de agente:  2
Tipo de calle:  2
Tipo de agente:  2
Tipo de calle:  3
Carro:  3605  - idTypeOfCar:  1
Coordinate position of the car:  (0, 30)
Numbers 